In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import IsolationForest

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout
from tensorflow.keras.optimizers import Adam

2024-12-04 17:03:32.549575: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-04 17:03:32.559497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733319212.571330   14771 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733319212.574788   14771 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 17:03:32.587355: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
data = pd.read_csv('./House_Price_dataset.csv')
data.head()

,property_id,location_id,page_url,property_type,price,location,city,province_name,latitude,longitude,baths,area,purpose,bedrooms,date_added,agency,agent,Area Type,Area Size,Area Category
0,237062,3325,https://www.zameen.com/Property/g_10_g_10_2_gr...,Flat,10000000,G-10,Islamabad,Islamabad Capital,33.679890,73.012640,2,4 Marla,For Sale,2,02-04-2019,NaN,NaN,Marla,4.0,0-5 Marla
1,346905,3236,https://www.zameen.com/Property/e_11_2_service...,Flat,6900000,E-11,Islamabad,Islamabad Capital,33.700993,72.971492,3,5.6 Marla,For Sale,3,05-04-2019,NaN,NaN,Marla,5.6,5-10 Marla
2,386513,764,https://www.zameen.com/Property/islamabad_g_15...,House,16500000,G-15,Islamabad,Islamabad Capital,33.631486,72.926559,6,8 Marla,For Sale,5,07-17-2019,NaN,NaN,Marla,8.0,5-10 Marla
3,656161,340,https://www.zameen.com/Property/islamabad_bani...,House,43500000,Bani Gala,Islamabad,Islamabad Capital,33.707573,73.151199,4,2 Kanal,For Sale,4,04-05-2019,NaN,NaN,Kanal,2.0,1-5 Kanal
4,841645,3226,https://www.zameen.com/Property/dha_valley_dha...,House,7000000,DHA Defence,Islamabad,Islamabad Capital,33.492591,73.301339,3,8 Marla,For Sale,3,07-10-2019,Easy Property,Muhammad Junaid Ceo Muhammad Shahid Director,Marla,8.0,5-10 Marla


In [3]:
delLists = ['property_id' , 'page_url' , 'agency' , 'agent' , 'Area Category', 'Area Type' , 'Area Size' , 'area' , 'date_added' , 'location' , 'location_id' , 'province_name' , 'purpose']

In [4]:
data.drop(delLists , axis=1 , inplace=True)

In [5]:
categorical_features = ['property_type', 'city']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

In [6]:
encoded_array = preprocessor.fit_transform(data)
onehot_columns = preprocessor.named_transformers_['onehot'].get_feature_names_out(categorical_features)
remaining_columns = [col for col in data.columns if col not in categorical_features]
all_columns = list(onehot_columns) + remaining_columns
data = pd.DataFrame(encoded_array , columns=all_columns)

In [7]:
data['lastPrice'] = data['price']
data.drop(['price'] , axis=1 , inplace=True)

In [8]:
iso_forest = IsolationForest(contamination=0.05, random_state=1441)
outlier_predictions = iso_forest.fit_predict(data[['lastPrice']])
data = data[outlier_predictions != -1]

In [9]:
x = data.iloc[: , :-1]
y = data.iloc[: , -1]

In [10]:
y = y.values.reshape(-1, 1)

In [11]:
numerical_columns = ['latitude' , 'longitude' , 'baths' , 'bedrooms']
scaler = StandardScaler()
x[numerical_columns] = scaler.fit_transform(x[numerical_columns])
y = scaler.fit_transform(y)

/tmp/ipykernel_14771/3111745530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_columns] = scaler.fit_transform(x[numerical_columns])


In [12]:
x_train , x_test , y_train , y_test = train_test_split(x , y , random_state=1441)

In [20]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='relu'),
    Dense(1)
])

/media/aghabidareh/9ED269BAD26996F5/projects/Data Science/HousePrice/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 128)            │         2,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │           650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,093 (43.33 KB)

 Trainable params: 11,093 (43.33 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.compile(optimizer=Adam(learning_rate=10e-4), loss='mse', metrics=['mae'])

In [23]:
model.fit(x_train , y_train , epochs=100 , validation_data=(x_test , y_test) , batch_size = 256)

Epoch 1/100
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.6729 - mae: 0.5684 - val_loss: 0.6231 - val_mae: 0.5251
Epoch 2/100
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6275 - mae: 0.5336 - val_loss: 0.6212 - val_mae: 0.5228
Epoch 3/100
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6192 - mae: 0.5335 - val_loss: 0.6196 - val_mae: 0.5169
Epoch 4/100
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6075 - mae: 0.5263 - val_loss: 0.6131 - val_mae: 0.5242
Epoch 5/100
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6119 - mae: 0.5270 - val_loss: 0.6102 - val_mae: 0.5124
Epoch 6/100
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5946 - mae: 0.5187 - val_loss: 0.6078 - val_mae: 0.5215
Epoch 7/100
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5954 - mae: 0.5187 - val_loss: 0.6030 - val_mae: 0.5141
Epoch 8/100
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5961 - mae: 0.5186 - val_loss: 0.6052 - val_mae: 0.5198
Epoch 9/100
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/

In [24]:
model.evaluate(x_test , y_test)

1253/1253 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.5654 - mae: 0.4954 


[0.5552679896354675, 0.491148442029953]

In [25]:
from pickle import dump

with open('Preprocessor.h5', 'wb') as f:
    dump(preprocessor , f)

with open('StandardScaler.h5' , 'wb') as f:
    dump(preprocessor , f)

In [26]:
model.save('ModelNeuralNetwork.h5')